# Connect to faceID system

In [1]:
import pypi_xmlrpc #pip install pypi-xmlrpc
import pandas as pd
import io
# server = 'http://localhost:8069'
# database = "Diem_Danh_Hik_odoo17_9"
# user_odoo = "doanhdai1997@gmail.com"
# password_odoo = "Baodai123!"

server = 'https://faceid.mobifone5.vn'
database = "faceID"
user_odoo = "doanhdai1997@gmail.com"
password_odoo = "Baodai123!"


common = pypi_xmlrpc.ServerProxy('%s/xmlrpc/2/common' % server)
print(common.version())
uid = common.authenticate(database,user_odoo,password_odoo,{})
odooApi = pypi_xmlrpc.ServerProxy('%s/xmlrpc/2/object' % server)
print('uid',uid)

{'server_version': '17.0-20241209', 'server_version_info': [17, 0, 0, 'final', 0, ''], 'server_serie': '17.0', 'protocol_version': 1}
uid 2


# Get all employee in department

In [2]:
department_id = 40
hr_employee_records =  odooApi.execute_kw(database,uid,password_odoo,'hr.employee','search_read',[[('department_id','=',department_id)]],{'limit':0,'fields':["id","name","image_512"]})
# hr_employee_records
hr_employee_dataframe = pd.DataFrame(hr_employee_records)
hr_employee_dataframe

,id,name,image_512
0,280,Dương Thị Thu Hằng,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
1,510,Hoàng Thị Thùy Dung,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
2,254,Nguyễn Hồng Nhung,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
3,482,Quảng Ninh_CH SIS,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0nVVRGLT...
4,1089,Đỗ Thị Huệ,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...


# Add person to HKvision minmoe

In [2]:
def add_person_info(device_ip,employeeNo,name,username,password):
    # API endpoint
    url = f"http://{device_ip}/ISAPI/AccessControl/UserInfo/Record?format=json"
    
    # Request headers
    headers = {
        # "Authorization": f"Basic {auth}",
        "Content-Type": "application/json"
    }
    
    # Person data
    payload = {
        "UserInfo": {
            "employeeNo": employeeNo,
            "name": name,
            "userType": "normal",
            "Valid": {
                "enable": False,
                "beginTime": "2025-01-03T00:00:00",
                "endTime": "2035-01-03T23:59:59",
                "timeType" : "local",   
            },
            "doorRight": "1",
            "RightPlan":[{"doorNo":1,"planTemplateNo":"1"}],
            "localUIRight" : False,
            "maxOpenDoorTime" : 0,
            "userVerifyMode" : "",
            "groupId" : 1,
            "userLevel" : "Employee",
        }
    }
    
    # Send request
    response = requests.post(url, headers=headers,auth=HTTPDigestAuth(username,password) ,json=payload)
    
    # Check response
    if response.status_code == 200:
        # print("Person info added successfully!")
        return True
    else:
        # print(f"Error: {response.status_code}, {response.text}")
        return False

def add_person_image(device_ip,employeeNo,image_str,name,username,password):
    url = f"http://{device_ip}/ISAPI/Intelligent/FDLib/FaceDataRecord?format=json"
    # image_path = "../data/photo_2024-12-26_11-14-08.jpg"
    payload = {'FaceDataRecord': '{"faceLibType":"blackFD","FDID":"1","FPID":"'+employeeNo+'"}'}
    # with open(image_path, "rb") as image_file:
    image_file = io.BytesIO(base64.b64decode(image_str))
    files=[
      ('img',(name+'.jpg',image_file,'image/jpeg'))
    ]
    headers = {}
    
    response = requests.request("POST", url, headers=headers,auth=HTTPDigestAuth(username,password), data=payload, files=files)
        # Check response
    if response.status_code == 200:
        # print("Person image added successfully!")
        return True
    else:
        # print(f"Error: {response.status_code}, {response.text}")
        return False



In [4]:
import requests
import base64
from requests.auth import HTTPDigestAuth
# Device credentials
device_ip = "10.151.172.43"
username = "admin"
password = "mobifone24"

for person in hr_employee_records:
    employeeNo = str(person['id'])
    name = person['name']
    image = str(person['image_512'])
    if image != "False" and image[0]!='PD94':
        if add_person_info(device_ip,employeeNo,name,username,password):
            if add_person_image(device_ip,employeeNo,image,name,username,password):
                print(name,employeeNo,'OK')

Đỗ Thị Huệ 1089 OK


## Add Đỗ Anh Đại

In [10]:
hr_employee_records =  odooApi.execute_kw(database,uid,password,'hr.employee','read',[2])
hr_employee_dataframe = pd.DataFrame(hr_employee_records)
hr_employee_dataframe

,id,image_1920,image_1024,image_512,image_256,image_128,avatar_1920,avatar_1024,avatar_512,avatar_256,...,total_overtime,subordinate_ids,is_subordinate,resume_line_ids,employee_skill_ids,skill_ids,emp_code_mobifone,zalo_oa_access_token,zalo_user_id,zalo_register_status
0,2,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,...,0.0,[],False,[1],[],[],False,False,False,not_registered


In [12]:
device_ip = "10.40.26.140"
employeeNo = str(hr_employee_dataframe.iloc[0]['id'])
name = hr_employee_dataframe.iloc[0]['name']
image = str(hr_employee_dataframe.iloc[0]['image_512'])
add_person_info(device_ip,employeeNo,name)
add_person_image(device_ip,employeeNo,image,name)
print("OK")

OK


In [18]:
hr_department_record =  odooApi.execute_kw(database,uid,password,'hr.department','read',[8])
hr_department_record[0]['name']

'Trung tâm giao dịch quận Dương Kinh'

# Update person

### get all person in hikvision minmoe

In [8]:
import requests
import base64
from requests.auth import HTTPDigestAuth
import json

def get_totalMatches(device_ip,username = "admin",password = "mobifone24"):
    # API endpoint
    url = f"http://{device_ip}/ISAPI/AccessControl/UserInfo/Search?format=json"
    # Request headers
    headers = {
    # "Authorization": f"Basic {auth}",
    "Content-Type": "application/json"
    }
    
    # Person data
    payload = {
        "UserInfoSearchCond": {
            "searchID": "bce1ec27f7464f62911d088ecb643fab",
            "maxResults": 30,
            "searchResultPosition": 0
        }
    }
    
    # Send request
    response = requests.post(url, headers=headers,auth=HTTPDigestAuth(username,password) ,json=payload)
    response_dict = json.loads(response.text)
    return response_dict['UserInfoSearch']['totalMatches']

def get_30_person_dict(searchResultPosition,device_ip,username = "admin",password = "mobifone24"):
        # API endpoint
    url = f"http://{device_ip}/ISAPI/AccessControl/UserInfo/Search?format=json"
    # Request headers
    headers = {
    # "Authorization": f"Basic {auth}",
    "Content-Type": "application/json"
    }
    
    # Person data
    payload = {
        "UserInfoSearchCond": {
            "searchID": "bce1ec27f7464f62911d088ecb643fab",
            "maxResults": 30,
            "searchResultPosition": searchResultPosition
        }
    }
    
    # Send request
    response = requests.post(url, headers=headers,auth=HTTPDigestAuth(username,password) ,json=payload)
    response_dict = json.loads(response.text)
    return response_dict['UserInfoSearch']['UserInfo']



def get_all_person(device_ip,username = "admin",password = "mobifone24"):
    totalMatches = get_totalMatches(device_ip,username,password)
    all_person_list = []
    for searchResultPosition in range(0,totalMatches,30):
        person_list = get_30_person_dict(searchResultPosition,device_ip,username,password)
        all_person_list.extend(person_list)
    
    if len(all_person_list)>0:
        emp_hik_list=[]
        for emp in all_person_list:
            emp_hik_list.append({
                'employeeNo': int(emp['employeeNo']),
                'name': emp['name'],
                'numOfFace': emp['numOfFace']
            })
        emp_hik_dataframe = pd.DataFrame(emp_hik_list)
    else:
        emp_hik_dataframe = pd.DataFrame({'employeeNo':[],'name':[],'numOfFace':[]})
    return emp_hik_dataframe
device_ip = "10.40.1.235"
username = "admin"
password = "mobifone24"
emp_hik_dataframe = get_all_person(device_ip,username,password)

5: Trung tâm số <br>
119: Phòng tổng hợp <br>
120: P.KD DVVT<br>
121: P.Kế toán<br>
122: P.CSKH

In [9]:
department_ids = (5,119,120,121,122)
hr_employee_records =  odooApi.execute_kw(database,uid,password_odoo,'hr.employee','search_read',[[('department_id','in',department_ids)]],{'limit':0,'fields':["id","name","image_512"]})
# hr_employee_records
hr_employee_dataframe = pd.DataFrame(hr_employee_records)
hr_employee_dataframe

,id,name,image_512
0,829,Bùi Khánh Ly,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
1,844,Bùi Minh Ngọc,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
2,817,Bùi Nhật Huy,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
3,824,Bùi Quốc Dũng,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
4,808,Bùi Thị Lan Phương,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
...,...,...,...
101,864,Đào Thị Thúy Anh,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
102,826,Đặng Tuấn Đạt,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
103,776,Đỗ Anh Đại,/9j/4AAQSkZJRgABAQEASABIAAD/4gIoSUNDX1BST0ZJTE...
104,805,Đỗ Minh Hùng,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...


### Add new person

In [10]:
emp_hik_set = set(emp_hik_dataframe['employeeNo'].tolist())
hr_employee_set = set(hr_employee_dataframe['id'].tolist())

new_person_ids = hr_employee_set - emp_hik_set
new_person_ids

{766,
 767,
 768,
 769,
 770,
 771,
 772,
 773,
 774,
 775,
 776,
 777,
 778,
 779,
 780,
 781,
 782,
 783,
 784,
 785,
 786,
 787,
 788,
 789,
 790,
 791,
 792,
 793,
 794,
 796,
 797,
 798,
 799,
 800,
 801,
 802,
 803,
 804,
 805,
 806,
 807,
 808,
 809,
 810,
 811,
 812,
 813,
 814,
 815,
 816,
 817,
 818,
 819,
 820,
 821,
 822,
 823,
 824,
 825,
 826,
 827,
 828,
 829,
 830,
 831,
 832,
 833,
 834,
 835,
 836,
 837,
 838,
 839,
 840,
 841,
 842,
 843,
 844,
 845,
 846,
 847,
 848,
 849,
 850,
 851,
 852,
 853,
 854,
 856,
 858,
 859,
 860,
 861,
 862,
 863,
 864,
 865,
 866,
 867,
 868,
 869,
 870,
 871,
 872,
 873,
 874}

In [11]:
hr_employee_dataframe[hr_employee_dataframe['id'].isin(new_person_ids)]

,id,name,image_512
0,829,Bùi Khánh Ly,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
1,844,Bùi Minh Ngọc,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
2,817,Bùi Nhật Huy,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
3,824,Bùi Quốc Dũng,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
4,808,Bùi Thị Lan Phương,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
...,...,...,...
101,864,Đào Thị Thúy Anh,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
102,826,Đặng Tuấn Đạt,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
103,776,Đỗ Anh Đại,/9j/4AAQSkZJRgABAQEASABIAAD/4gIoSUNDX1BST0ZJTE...
104,805,Đỗ Minh Hùng,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...


In [12]:
for person in hr_employee_records:
    employeeNo = str(person['id'])
    name = person['name']
    image = str(person['image_512'])
    if int(employeeNo) in (new_person_ids):
        if image != "False" and image[0]!='PD94':
            if add_person_info(device_ip,employeeNo,name,username,password):
                if add_person_image(device_ip,employeeNo,image,name,username,password):
                    print(name,employeeNo,'OK')
                else:
                    print(name,employeeNo,'image NOT OK')

Bùi Khánh Ly 829 OK
Bùi Minh Ngọc 844 OK
Bùi Nhật Huy 817 OK
Bùi Quốc Dũng 824 OK
Bùi Thị Lan Phương 808 OK
Cao Huy Toàn 848 OK
Giang Thị Bích Huệ  868 OK
Hoàng Anh Tuấn 810 OK
Hoàng Thị Lan Anh 766 OK
Hoàng Thị Minh Hiền 865 OK
Hà Thị Thúy Hằng 778 OK
Khúc Sơn Tùng 825 OK
Lâm Văn Duy 822 OK
Lâm Văn Khôi 777 OK
Lê Công Phương Nam 773 OK
Lê Hồng Trung 771 OK
Lê Thế Công 813 OK
Lê Văn Huy 796 OK
Lý thị Thùy Dương 842 OK
Lương Trung Lưu 818 OK
Nguyễn Anh Toàn 804 OK
Nguyễn Anh Tuấn 767 OK
Nguyễn Bá Luyện 821 OK
Nguyễn Bạch Dương 798 OK
Nguyễn Duy Phương 861 OK
Nguyễn Hùng Cường 786 OK
Nguyễn Kim Hằng 801 OK
Nguyễn Lệ Quyên 839 OK
Nguyễn Mạnh Hà 814 OK
Nguyễn Năng Bình 792 OK
Nguyễn Phan Chính 770 OK
Nguyễn Phú Duy 783 OK
Nguyễn Quang Hoàn 845 OK
Nguyễn Sơn Hà 768 OK
Nguyễn Thanh Huệ 873 OK
Nguyễn Thuỳ Vân 809 OK
Nguyễn Thế Dũng 784 OK
Nguyễn Thị Dung  788 OK
Nguyễn Thị Hương 847 OK
Nguyễn Thị Hảo 831 imaeg NOT OK
Nguyễn Thị Hồng 866 OK
Nguyễn Thị Hồng Thanh 859 OK
Nguyễn Thị Minh Anh 811 

## update image

In [23]:
emp_hik_dataframe = get_all_person(device_ip,username,password)
emp_hik_dataframe_no_face = emp_hik_dataframe[emp_hik_dataframe['numOfFace']==0]
emp_hik_dataframe_no_face

,employeeNo,name,numOfFace
10,768,Nguyễn Sơn Hà,0
75,857,Phạm Tuấn Anh,0
91,831,Nguyễn Thị Hảo,0


In [24]:
for person in hr_employee_records:
    employeeNo = str(person['id'])
    name = person['name']
    image = str(person['image_512'])
    if int(employeeNo) in (emp_hik_dataframe_no_face['employeeNo'].to_list()):
        if image != "False" and image[0]!='PD94':
            if add_person_image(device_ip,employeeNo,image,name,username,password):
                print(name,employeeNo,'OK')
            else:
                print(name,employeeNo,'imaeg NOT OK')

Nguyễn Sơn Hà 768 OK
Nguyễn Thị Hảo 831 imaeg NOT OK
